In [ ]:
!pip install wheel
!pip install pipwin

!pipwin install numpy
!pipwin install pandas
!pipwin install shapely
!pipwin install gdal
!pipwin install fiona
!pipwin install pyproj
!pipwin install six
!pipwin install rtree
!pipwin install geopandas

  Stored in directory: C:\Users\kenda\AppData\Local\pip\Cache\wheels\ed\b5\a9\f8bf742bfd2a45bea5b27368a5e3e5a390cda636d40829a1fc
  Stored in directory: C:\Users\kenda\AppData\Local\pip\Cache\wheels\9b\04\dd\7daf4150b6d9b12949298737de9431a324d4b797ffd63f526e
  Stored in directory: C:\Users\kenda\AppData\Local\pip\Cache\wheels\a2\73\e6\3e433f3fd78257c3f971baf8cc9001cc0c4797268c61751e89
Successfully built pipwin docopt pyjsparser
  Found existing installation: beautifulsoup4 4.7.1
    Uninstalling beautifulsoup4-4.7.1:
      Successfully uninstalled beautifulsoup4-4.7.1


In [3]:
!pip install folium

In [5]:
!pip install shapely

In [32]:
import pandas as pd
import altair as alt
# import geopandas
import folium
import matplotlib.pyplot as plt
import shapely

### Import Data from BigQuery for SF Bike Share

In [12]:
# june july august sept 2017 trips
JJAS_trips = pd.read_csv('sf_bikesharetrips_2017_junjulaugsept.csv')

# october november december 2017 trips
OND_trips = pd.read_csv('sf_bikesharetrips_2017_octnovdec.csv')

# SF bikeshare regions
regions = pd.read_csv("sf_bikeshare_regions.csv")

# SF bikeshare stations
station_info = pd.read_csv('sf_bikeshare_station_info.csv')

## Concatenate the 2 trip dataframes

In [13]:
sf_bikeshare_trips = pd.concat([JJAS_trips, OND_trips], axis = 0)

In [14]:
sf_bikeshare_trips.describe()

,duration_sec,start_station_id,end_station_id,bike_number,zip_code,start_station_latitude,start_station_longitude,end_station_latitude,end_station_longitude,member_birth_year,bike_share_for_all_trip
count,519700.000000,519700.000000,519700.000000,519700.000000,0.0,519700.000000,519700.000000,519700.000000,519700.000000,453159.000000,0.0
mean,1099.009521,95.034245,92.184041,1672.533079,NaN,37.771653,-122.363927,37.771844,-122.363236,1980.404787,NaN
std,3444.146451,86.083078,84.969491,971.356959,NaN,0.086305,0.105573,0.086224,0.105122,10.513488,NaN
min,61.000000,3.000000,3.000000,10.000000,NaN,37.317298,-122.444293,37.317298,-122.444293,1886.000000,NaN
25%,382.000000,24.000000,23.000000,787.000000,NaN,37.773492,-122.411726,37.774520,-122.410345,1974.000000,NaN
50%,596.000000,67.000000,66.000000,1728.500000,NaN,37.783521,-122.398870,37.783830,-122.398525,1983.000000,NaN
75%,938.000000,139.000000,134.000000,2520.000000,NaN,37.795392,-122.391034,37.795392,-122.391034,1988.000000,NaN
max,86369.000000,340.000000,340.000000,3733.000000,NaN,37.880222,-121.874119,37.880222,-121.874119,1999.000000,NaN


In [15]:
sf_bikeshare_trips.columns

Index(['trip_id', 'duration_sec', 'start_date', 'start_station_name',
       'start_station_id', 'end_date', 'end_station_name', 'end_station_id',
       'bike_number', 'zip_code', 'subscriber_type', 'c_subscription_type',
       'start_station_latitude', 'start_station_longitude',
       'end_station_latitude', 'end_station_longitude', 'member_birth_year',
       'member_gender', 'bike_share_for_all_trip', 'start_station_geom',
       'end_station_geom'],
      dtype='object')

In [16]:
sf_bikeshare_trips.dtypes

trip_id                     object
duration_sec                 int64
start_date                  object
start_station_name          object
start_station_id             int64
end_date                    object
end_station_name            object
end_station_id               int64
bike_number                  int64
zip_code                   float64
subscriber_type             object
c_subscription_type         object
start_station_latitude     float64
start_station_longitude    float64
end_station_latitude       float64
end_station_longitude      float64
member_birth_year          float64
member_gender               object
bike_share_for_all_trip    float64
start_station_geom          object
end_station_geom            object
dtype: object

In [17]:
sf_bikeshare_trips['start_date'] = pd.to_datetime(sf_bikeshare_trips['start_date'])

In [18]:
sf_bikeshare_trips['end_date'] = pd.to_datetime(sf_bikeshare_trips['end_date'])

### # Trips Taken By Week

In [20]:
ridesbyweek = pd.DataFrame(sf_bikeshare_trips.groupby(pd.Grouper(key='start_date',freq='W'))["trip_id"].count()).reset_index()

In [21]:
alt.Chart(ridesbyweek).mark_line().encode(
    x='start_date',
    y=alt.Y('trip_id', title = "Trip Counts")
)

alt.Chart(...)

### Trips by Subscriber Type (Customer vs. Subscriber)

In [83]:
ridesbysubscriber = sf_bikeshare_trips.groupby([pd.Grouper(key='start_date',freq='W'), 'subscriber_type'])\
                                                ["trip_id"].count().reset_index()

In [84]:
ridesbysubscriber.head()

,start_date,subscriber_type,trip_id
0,2017-07-02 00:00:00+00:00,Customer,1106
1,2017-07-02 00:00:00+00:00,Subscriber,2581
2,2017-07-09 00:00:00+00:00,Customer,1877
3,2017-07-09 00:00:00+00:00,Subscriber,3879
4,2017-07-16 00:00:00+00:00,Customer,2699


In [85]:
alt.Chart(ridesbysubscriber, title= "Ride Trends by Subscriber Type").mark_line().encode(
    x='start_date',
    y=alt.Y('trip_id', title = "Trip Counts"),
    color='subscriber_type'
)

alt.Chart(...)

### Average Ride Duration by Subsriber Type (seconds)

In [48]:
ridesbylength = sf_bikeshare_trips.groupby([pd.Grouper(key='start_date',freq='W'), 'subscriber_type'])\
                                                ["duration_sec"].mean().reset_index()

In [70]:
alt.Chart(ridesbysubscriber, title = "Average Ride Time (seconds) by Subscriber Type").mark_line().encode(
    x='start_date',
    y=alt.Y('duration_sec', title = "Average Duration"),
    color='subscriber_type'
)

alt.Chart(...)

### Age/Gender Distribution

In [61]:
#filter out data before 1917, no one over 100 years old is likely riding a bike

age_gender = pd.DataFrame(sf_bikeshare_trips[sf_bikeshare_trips["member_birth_year"]>1917].groupby(
                ["member_birth_year", "member_gender"])["trip_id"].count()).reset_index()

In [69]:
alt.Chart(age_gender, title = "Number of Rides in 2017 by Gender and Birth Year").mark_line().encode(
    x='member_birth_year:O',
    y=alt.Y('trip_id', title = "Number Rides"),
    color='member_gender:N'
)

alt.Chart(...)

### Busiest Hour of Day

In [124]:
sf_bikeshare_trips["start_hour"] = sf_bikeshare_trips["start_date"].dt.hour
sf_bikeshare_trips["end_hour"] = sf_bikeshare_trips["end_date"].dt.hour

In [125]:
start_hour = sf_bikeshare_trips.groupby("start_hour")["trip_id"].count().reset_index()
end_hour = sf_bikeshare_trips.groupby("end_hour")["trip_id"].count().reset_index()

In [127]:
start = alt.Chart(start_hour, title = "Most Popular Start Hour").mark_bar().encode(
    x='start_hour:O',
    y=alt.Y('trip_id', title = "Number Rides")
)

end = alt.Chart(end_hour, title = "Most Popular End Hour").mark_bar().encode(
    x='end_hour:O',
    y=alt.Y('trip_id', title = "Number Rides")
)

start|end

alt.HConcatChart(...)

### Top 10 Most Popular Bikes

In [81]:
sf_bikeshare_trips['bike_number'].value_counts().sort_values(ascending = False).head(10)

68      457
2178    426
210     408
813     403
602     402
775     399
256     396
796     396
395     396
190     395
Name: bike_number, dtype: int64

### Most Common Station Pairs

In [75]:
pd.DataFrame(sf_bikeshare_trips.groupby(["start_station_name", "end_station_name"])["trip_id"].count().sort_values(ascending=False))

trip_id
start_station_name                                 end_station_name                                           
San Francisco Ferry Building (Harry Bridges Plaza) The Embarcadero at Sansome St                          3344
The Embarcadero at Sansome St                      Steuart St at Market St                                1793
                                                   San Francisco Ferry Building (Harry Bridges Plaza)     1734
Berry St at 4th St                                 San Francisco Ferry Building (Harry Bridges Plaza)     1686
2nd St at S Park St                                Montgomery St BART Station (Market St at 2nd St)       1509
Montgomery St BART Station (Market St at 2nd St)   2nd St at S Park St                                    1485
San Francisco Ferry Building (Harry Bridges Plaza) Berry St at 4th St                                     1401
Steuart St at Market St                            The Embarcadero at Sansome St                          1361
2nd St at Townsend St - Coming Soon                San Francisco Ferry Building (Harry Bridges Plaza)     1274
19th Street BART Station                           Grand Ave at Perkins St                                1196
The Embarcadero at Sansome St                      The Embarcadero at Sansome St                          1190
Howard St at Beale St                              San Francisco Caltrain (Townsend St at 4th St)         1183
Market St at 10th St                               San Francisco Caltrain Station 2  (Townsend St ...     1148
San Francisco Ferry Building (Harry Bridges Plaza) 2nd St at Townsend St - Coming Soon                    1105
Howard St at 2nd St                                San Francisco Caltrain (Townsend St at 4th St)         1100
Bay Pl at Vernon St                                19th Street BART Station                               1054
Grand Ave at Perkins St                            19th Street BART Station                               1041
The Embarcadero at Steuart St                      San Francisco Caltrain (Townsend St at 4th St)         1037
5th St at Howard St                                San Francisco Caltrain Station 2  (Townsend St ...     1031
Montgomery St BART Station (Market St at 2nd St)   San Francisco Caltrain (Townsend St at 4th St)          987
Embarcadero BART Station (Beale St at Market St)   Beale St at Harrison St                                 939
Townsend St at 7th St                              San Francisco Caltrain Station 2  (Townsend St ...      912
19th Street BART Station                           Bay Pl at Vernon St                                     897
3rd St at Townsend St                              San Francisco Ferry Building (Harry Bridges Plaza)      893
Steuart St at Market St                            Berry St at 4th St                                      848
Market St at 10th St                               Powell St BART Station (Market St at 4th St)            839
2nd St at Townsend St - Coming Soon                Montgomery St BART Station (Market St at 2nd St)        794
Steuart St at Market St                            2nd St at Townsend St - Coming Soon                     792
San Francisco Ferry Building (Harry Bridges Plaza) San Francisco Ferry Building (Harry Bridges Plaza)      788
Bancroft Way at College Ave                        Downtown Berkeley BART                                  776
...                                                                                                        ...
Townsend St at 5th St                              Koshland Park                                             1
DeFremery Park                                     Webster St at 17th St                                     1
Shattuck Ave at 51st St                            Franklin St at 9th St                                     1
MLK Jr Way at University Ave                       Shattuck Ave at 55th Ave                                

### Map of Stations

In [33]:
sf_bikeshare_trips.columns

Index(['trip_id', 'duration_sec', 'start_date', 'start_station_name',
       'start_station_id', 'end_date', 'end_station_name', 'end_station_id',
       'bike_number', 'zip_code', 'subscriber_type', 'c_subscription_type',
       'start_station_latitude', 'start_station_longitude',
       'end_station_latitude', 'end_station_longitude', 'member_birth_year',
       'member_gender', 'bike_share_for_all_trip', 'start_station_geom',
       'end_station_geom'],
      dtype='object')

In [42]:
map = folium.Map(location = [37.71,-122.28], tiles='OpenStreetMap' , zoom_start = 9)
map

In [41]:
geo_df_list = [[point.xy[1][0], point.xy[0][0]] for point in start_station_geom.start_station_geom]

AttributeError: 'str' object has no attribute 'xy'